### Live trade execution script

To run this script, just make sure you have downloaded and are logged in to your IBKR TWS account

In [1]:
%pip install ib_insync


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pyts.image import GramianAngularField
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
import threading
import time
from datetime import datetime
from ib_insync import IB, util, Stock
import pandas as pd

In [3]:
# === Load trained CNN model and candlestick class mappings ===
model = tf.keras.models.load_model("/Users/alfredbidokwu/Documents/ML-Finance/PGMP_15/1min_masterpiece_densenet_finetuned_v2.keras")
class_names = ['Bearish Engulfing', 'Bullish Engulfing', 'Hammer', 'Hanging Man',
               'No Pattern', 'Piercing Line', 'Shooting Star', 'Tweezer Bottom', 'Tweezer Top']

# Mapping detected patterns to trading signals (long, short, or flat)
pattern_to_signal = {
    "Tweezer Bottom": "Long",
    "Piercing Line": "Long",
    "Bullish Engulfing": "Long",
    "Hammer": "Long",
    "Tweezer Top": "Short",
    "Bearish Engulfing": "Short",
    "Hanging Man": "Short",
    "Shooting Star": "Short",
    "No Pattern": "Flat"
}


# === Interactive Brokers (IB) API Setup ===
class IBWrapper(EWrapper):
    """Handles all the IB API callbacks like historical data, errors, order IDs etc."""
    def __init__(self):
        super().__init__()
        self.data = []  # stores historical bar data
        self.next_order_id = None  # will be set once we receive it from IB
        self.historical_data_end = False  # flag to track if historical data pull is complete

    def historicalData(self, reqId, bar):
        """Each bar received gets appended here."""
        self.data.append({
            "date": bar.date,
            "open": bar.open,
            "high": bar.high,
            "low": bar.low,
            "close": bar.close,
            "volume": bar.volume
        })

    def historicalDataEnd(self, reqId, start, end):
        """IB notifies us when historical data request is finished."""
        self.historical_data_end = True

    def nextValidId(self, orderId: int):
        """Receives and stores the next valid order ID from IB."""
        self.next_order_id = orderId

    def error(self, reqId, errorCode, errorString):
        """Print any errors IB sends."""
        print(f"Error: {errorCode} - {errorString}")


class IBClient(EClient):
    """Handles sending requests to IB."""
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)

    def request_latest_data(self, contract):
        """Ask IB for latest 1-minute historical bars for a given contract."""
        self.reqHistoricalData(
            reqId=1,
            contract=contract,
            endDateTime="",
            durationStr="1 D",
            barSizeSetting="1 min",
            whatToShow="MIDPOINT",
            useRTH=1,
            formatDate=1,
            keepUpToDate=False,
            chartOptions=[]
        )


def create_contract(symbol, sec_type="CMDTY", exchange="SMART", currency="USD"):
    """Creates an IB contract object for the symbol we want to trade."""
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.exchange = exchange
    contract.currency = currency
    return contract

def create_order(order_id, action, quantity, order_type="MKT"):
    """Creates a basic market order."""
    order = Order()
    order.orderId = order_id
    order.action = action  # BUY or SELL
    order.totalQuantity = quantity
    order.orderType = order_type
    order.eTradeOnly = False
    order.firmQuoteOnly = False
    return order


# === Rule-Based Pattern Detection (Backup for ML Model) ===
def detect_candlestick_pattern(current, previous):
    """Manually detects basic candlestick patterns based on OHLC values."""
    o, h, l, c = current
    prev_o, prev_h, prev_l, prev_c = previous
    
    # Simple tweezer logic
    if c > o and l == prev_l and c > prev_c:
        return "Tweezer Bottom"
    elif o > c and h == prev_h and c < prev_o:
        return "Tweezer Top"
    
    # Piercing line pattern
    elif c > o and o < prev_c and c > (prev_o + prev_c) / 2:
        return "Piercing Line"
    
    # More general bullish/bearish candles
    elif c > o:
        if (h - c) > 2 * (o - l):
            return "Shooting Star"
        elif (c - o) > 2 * (h - c):
            return "Bullish Engulfing"
        else:
            return "Hammer"
    elif c < o:
        if (c - l) > 2 * (h - c):
            return "Hanging Man"
        elif (o - c) > 2 * (l - c):
            return "Bearish Engulfing"
        else:
            return "Hammer"
    
    return "No Pattern"


# === Preprocessing for ML Model Inference ===
def convert_to_gaf_image(data_row):
    """Converts OHLCV data into a Gramian Angular Field (GAF) image."""
    ts = np.array([data_row['open'], data_row['high'], data_row['low'], data_row['close']])
    gaf = GramianAngularField(image_size=len(ts))  # create small 4x4 GAF matrix
    image = gaf.fit_transform([ts])[0]  # GAF output
    image = np.expand_dims(image, axis=-1)  # add channel dimension
    image = np.repeat(image, 3, axis=-1)    # make it RGB-like (4,4,3) for DenseNet
    return image


# === Trade Execution Based on Signals ===
def execute_trade(signal, wrapper, client, contract, quantity=1):
    """Places a market order if signal is Long or Short."""
    if signal == "Flat":
        print("No trade signal.")
        return

    if wrapper.next_order_id is None:
        print("❌ No valid order ID available. Skipping trade.")
        return

    action = "BUY" if signal == "Long" else "SELL"
    order_id = wrapper.next_order_id
    order = create_order(order_id, action, quantity)
    client.placeOrder(order_id, contract, order)
    print(f"💸 Executed {action} order based on signal: {signal}, order_id: {order_id}")

    # 🔥 Always increment order IDs after use to prevent IB API errors
    wrapper.next_order_id += 1



# === Main Real-Time Trading Loop ===
def start_trading_loop(symbol="XAUUSD", quantity=1):
    """This is the main live trading loop."""
    wrapper = IBWrapper()
    client = IBClient(wrapper)
    client.connect("127.0.0.1", 7497, clientId=2)  # connects to TWS or IB Gateway
    # client.reqIds(-1)  # optional: force IB to send us a fresh order ID

    threading.Thread(target=client.run, daemon=True).start()
    time.sleep(30)  # give IB client some time to fully boot up

    contract = create_contract(symbol)

    print("🚀 Starting real-time strategy loop...")

    try:
        while True:
            wrapper.data.clear()
            wrapper.historical_data_end = False
            client.request_latest_data(contract)

            # Wait until historical bars are fully received
            wait_time = 0
            while not wrapper.historical_data_end and wait_time < 10:
                time.sleep(2)
                wait_time += 2

            if not wrapper.data:
                print("No data fetched. Retrying...")
                time.sleep(60)
                continue

            current_data = wrapper.data[-1]  # last bar
            prev_data = wrapper.data[-2] if len(wrapper.data) >= 2 else current_data  # second to last bar

            # First do rule-based detection
            pattern = detect_candlestick_pattern(
                [current_data['open'], current_data['high'], current_data['low'], current_data['close']],
                [prev_data['open'], prev_data['high'], prev_data['low'], prev_data['close']]
            )

            # Then transform data and predict with CNN
            gaf_img = convert_to_gaf_image(current_data)
            gaf_img_resized = tf.image.resize(gaf_img, (96, 96))  # resize to model input size
            gaf_img_resized = tf.expand_dims(gaf_img_resized, axis=0)  # add batch dimension
            gaf_img_resized = gaf_img_resized / 255.0  # normalize pixel values

            prediction = model.predict(gaf_img_resized)
            predicted_class = class_names[np.argmax(prediction)]

            # Hybrid rule: trust model unless model says "No Pattern"// 
            # We do this optional step to cross validate the model's performance where the model predict's with low confidence.
            # If you do not want to use it please comment the if statement below and uncomment the signal = pattern_to_signal.get(predicted_class, "Flat")  portion below the else statement

            if predicted_class == "No Pattern" and pattern != "No Pattern":
                signal = pattern_to_signal.get(pattern, "Flat")
            else:
                signal = pattern_to_signal.get(predicted_class, "Flat")
            
            # signal = pattern_to_signal.get(predicted_class, "Flat")

            print(f"📈 Signal: {signal}")

            # Only trade if clear long/short signal
            if signal in ["Long", "Short"]:
                execute_trade(signal, wrapper, client, contract, quantity)

            print("⏳ Waiting for next candle...\n")
            time.sleep(60)  # wait till next 1-minute candle

    except KeyboardInterrupt:
        print("🛑 Trading stopped by user.")
        client.disconnect()


# === Fire it up! ===
start_trading_loop("XAUUSD", quantity=1)

Error: 2104 - Market data farm connection is OK:usfarm.nj
Error: 2104 - Market data farm connection is OK:cashfarm
Error: 2104 - Market data farm connection is OK:eufarmnj
Error: 2104 - Market data farm connection is OK:usfarm
Error: 2106 - HMDS data farm connection is OK:cashhmds
Error: 2106 - HMDS data farm connection is OK:ushmds
Error: 2158 - Sec-def data farm connection is OK:secdefil
🚀 Starting real-time strategy loop...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
📈 Signal: Long
💸 Executed BUY order based on signal: Long, order_id: 137
⏳ Waiting for next candle...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
📈 Signal: Short
💸 Executed SELL order based on signal: Short, order_id: 138
⏳ Waiting for next candle...

Error: 2108 - Market data farm connection is inactive but should be available upon demand.eufarmnj
Error: 2108 - Market data farm connection is inactive but should be available upon demand.eufarmnj
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
📈 Signal: Long
💸 Executed BUY order based on 